# Customer Behaviour Insight Engine - Data Cleaning

This notebook loads the raw e-commerce clickstream dataset and prepares it for analysis. 
Goals of this step:
- Verify that all source files load correctly
- Inspect basic shapes and schema
- Identify missing values and type issues
- Create a clear cleaning checklist for the next steps

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
data_path = "../data/raw/kaggle_clickstream_data/"

customers = pd.read_csv(data_path + "customers.csv")
products = pd.read_csv(data_path + "products.csv")
orders = pd.read_csv(data_path + "orders.csv")
order_items = pd.read_csv(data_path + "order_items.csv")
events = pd.read_csv(data_path + "events.csv")
sessions = pd.read_csv(data_path + "sessions.csv")
reviews = pd.read_csv(data_path + "reviews.csv")

tables = {
    "customers": customers,
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "events": events, 
    "sessions": sessions, 
    "reviews": reviews,
}

for name, df in  tables.items():
    print(f"{name:12} -> shape = {df.shape}")

customers    -> shape = (20000, 7)
products     -> shape = (1197, 6)
orders       -> shape = (33580, 10)
order_items  -> shape = (59163, 5)
events       -> shape = (760958, 10)
sessions     -> shape = (120000, 6)
reviews      -> shape = (10780, 6)


## 1. Load check

All seven source tables are loaded from `data/raw/kaggle_clickstream_data/`:

- `customers`
- `products`
- `orders`
- `order_items`
- `events`
- `sessions`
- `reviews`

Next: inspect schema and missiong values to build a cleaning plan. 

In [9]:
# Quick schema + missing summary for key tables 

for name in ["events", "orders", "customers"]:
    df = tables[name]
    print(f"\n=== {name.upper()} ===")
    display(df.head(3))
    print("\n.info():")
    print(df.info())
    print("\nMissing values per column:")
    print(df.isnull().sum())
    print("-" * 60)


=== EVENTS ===


,event_id,session_id,timestamp,event_type,product_id,qty,cart_size,payment,discount_pct,amount_usd
0,1,1,2021-12-27T00:08:36,page_view,93.0,NaN,NaN,NaN,NaN,NaN
1,2,1,2021-12-27T00:16:36,page_view,1005.0,NaN,NaN,NaN,NaN,NaN
2,3,1,2021-12-27T00:18:01,add_to_cart,1005.0,1.0,NaN,NaN,NaN,NaN



.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760958 entries, 0 to 760957
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   event_id      760958 non-null  int64  
 1   session_id    760958 non-null  int64  
 2   timestamp     760958 non-null  object 
 3   event_type    760958 non-null  object 
 4   product_id    682469 non-null  float64
 5   qty           143126 non-null  float64
 6   cart_size     44909 non-null   float64
 7   payment       33580 non-null   object 
 8   discount_pct  33580 non-null   float64
 9   amount_usd    33580 non-null   float64
dtypes: float64(5), int64(2), object(3)
memory usage: 58.1+ MB
None

Missing values per column:
event_id             0
session_id           0
timestamp            0
event_type           0
product_id       78489
qty             617832
cart_size       716049
payment         727378
discount_pct    727378
amount_usd      727378
dtype: int64
------------

,order_id,customer_id,order_time,payment_method,discount_pct,subtotal_usd,total_usd,country,device,source
0,1,13917,2025-01-31T23:07:42,card,20,107.15,85.72,PL,desktop,organic
1,2,1022,2024-02-19T01:17:50,card,0,116.17,116.17,FR,tablet,organic
2,3,6145,2024-12-04T20:24:13,card,0,137.35,137.35,US,mobile,organic



.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33580 entries, 0 to 33579
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        33580 non-null  int64  
 1   customer_id     33580 non-null  int64  
 2   order_time      33580 non-null  object 
 3   payment_method  33580 non-null  object 
 4   discount_pct    33580 non-null  int64  
 5   subtotal_usd    33580 non-null  float64
 6   total_usd       33580 non-null  float64
 7   country         33580 non-null  object 
 8   device          33580 non-null  object 
 9   source          33580 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 2.6+ MB
None

Missing values per column:
order_id          0
customer_id       0
order_time        0
payment_method    0
discount_pct      0
subtotal_usd      0
total_usd         0
country           0
device            0
source            0
dtype: int64
---------------------------------

,customer_id,name,email,country,age,signup_date,marketing_opt_in
0,1,Jennifer Salinas,nicholas59@example.org,JP,71,2020-09-04,True
1,2,Phillip Ramos,christinarubio@example.com,IN,26,2020-04-05,False
2,3,Dawn Fowler,jessica03@example.org,BR,21,2023-08-31,True



.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       20000 non-null  int64 
 1   name              20000 non-null  object
 2   email             20000 non-null  object
 3   country           20000 non-null  object
 4   age               20000 non-null  int64 
 5   signup_date       20000 non-null  object
 6   marketing_opt_in  20000 non-null  bool  
dtypes: bool(1), int64(2), object(4)
memory usage: 957.2+ KB
None

Missing values per column:
customer_id         0
name                0
email               0
country             0
age                 0
signup_date         0
marketing_opt_in    0
dtype: int64
------------------------------------------------------------


## 3. Initial Data Cleaning Plan 

Based on the basic inspection of the three core tables (`events`, `orders`, `customers`), these are the required cleaning tasks:
### A. Convert all timestamp fields to proper datetime format
- `events.timestamp` (currently object/string)
- `orders.order_time`
- `customers.signup_date`
- These need conversion in order to sort chronologically and extract hour/day features.

---

### B. Document expected missiong values (do not fill them yet)

Missing values in `events` are expected behaviour:
- `product_id` missiong for checkout/purchase events
-  `qty` and `cart_size` only apply to add_to_cart events
-  `payment`, `discount_pct`, `amount_usd` only apply to purchase events

These will not be filled yet. They will be handled by event_type later. 

---

### C. Validate ID columns

- Ensure `customer_id` has no duplicates
- Ensure `event_id`, `order_id` and `session_id` are uique
- Later: confirm order_items links t valid products and valid orders

---

### D. Validate basic value ranges 

- No negative prices, quantites or totals
- Age should be within reasonable range (e.g. 18-100)
- Discount_pct between 0 and 100

---

### Next Step

Proceed with timestamp cleaning and feature extraction (hour, day of week).

  

In [12]:
#convert timestamp-like columns to datetime

events["timestamp"] = pd.to_datetime(events["timestamp"], errors="coerce")
orders["order_time"] = pd.to_datetime(orders["order_time"], errors="coerce")
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")

events.timestamp.head(), orders.order_time.head(), customers.signup_date.head()

(0   2021-12-27 00:08:36
 1   2021-12-27 00:16:36
 2   2021-12-27 00:18:01
 3   2021-12-27 00:45:36
 4   2021-12-27 01:03:36
 Name: timestamp, dtype: datetime64[ns],
 0   2025-01-31 23:07:42
 1   2024-02-19 01:17:50
 2   2024-12-04 20:24:13
 3   2024-07-17 08:50:47
 4   2020-08-21 16:54:16
 Name: order_time, dtype: datetime64[ns],
 0   2020-09-04
 1   2020-04-05
 2   2023-08-31
 3   2022-06-30
 4   2022-07-22
 Name: signup_date, dtype: datetime64[ns])

In [14]:
# Add time features for behaviour analysis

events["hour"] = events["timestamp"].dt.hour
events["day_of_week"] = events["timestamp"].dt.day_name()

events[["timestamp", "hour", "day_of_week"]].head()


,timestamp,hour,day_of_week
0,2021-12-27 00:08:36,0,Monday
1,2021-12-27 00:16:36,0,Monday
2,2021-12-27 00:18:01,0,Monday
3,2021-12-27 00:45:36,0,Monday
4,2021-12-27 01:03:36,1,Monday


## 4. Validate Expected Missing Values (Events Table)

The `events` table contains many missing values, but these are *expected* and refelct normal user behaviour. 

- `product_id` is missing for `checkout` and `purchase` events
- `qty` is present only for `add_to_cart`
- `cart_size` appears only during cart actions
-  `payment`, `discount_pct` and `amount_usd` appear only for `purchase` events.

Verify these patterns with grouped checks. 

In [15]:
# Check missing product_id by event_type 

events.groupby("event_type")["product_id"].apply(lambda x: x.isnull().sum())

event_type
add_to_cart        0
checkout       44909
page_view          0
purchase       33580
Name: product_id, dtype: int64

In [17]:
# Check missing qty by event_type

events.groupby("event_type")["qty"].apply(lambda x: x.isnull().sum())

event_type
add_to_cart         0
checkout        44909
page_view      539343
purchase        33580
Name: qty, dtype: int64

In [21]:
# Check missing payment fields by event_type

events.groupby("event_type")[["payment", "discount_pct", "amount_usd"]].apply(lambda x: x.isnull().sum())

,payment,discount_pct,amount_usd
event_type,,,
add_to_cart,143126,143126,143126
checkout,44909,44909,44909
page_view,539343,539343,539343
purchase,0,0,0


## 5. Validate ID Columns (Referential Integrity)

Here I check that all keys actually point to valid records: 

- Every `session_id` in `events` exists in `sessionss`
- Every `customer_id` in `sessions` exists in `customers`
- Every `customer_id` in `orders` exists in `customers`
- Every `order_id` and `product_id` in `order_items` exist in `orders` and `products`

If these checks return 0 "bad" rows, the relationships between tables are consistent and safe to use for joins.

In [23]:
# Validate event → session → customer links

# Events referencing non-existent sessions
bad_sessions = events[~events["session_id"].isin(sessions["session_id"])]
print("Events with invalid session_id:", len(bad_sessions))

# Sessions referencing non-existent customers
bad_customers_sessions = sessions[~sessions["customer_id"].isin(customers["customer_id"])]
print("Sessions with invalid customer_id:", len(bad_customers_sessions))

# Orders referencing non-existent customers
bad_customers_orders = orders[~orders["customer_id"].isin(customers["customer_id"])]
print("Orders with invalid customer_id:", len(bad_customers_orders))

Events with invalid session_id: 0
Sessions with invalid customer_id: 0
Orders with invalid customer_id: 0


In [25]:
# Validate order_items links

# Order items with invalid order_id
bad_order_ids = order_items[~order_items["order_id"].isin(orders["order_id"])]
print("Order items with invalid order_id:", len(bad_order_ids))

# Order items with invalid product_id
bad_product_ids = order_items[~order_items["product_id"].isin(products["product_id"])]
print("Order items with invalid product_id:", len(bad_product_ids))

# Optional: show examples if problems exist
if len(bad_order_ids) > 0:
    display(bad_order_ids.head())

if len(bad_product_ids) > 0:
    display(bad_product_ids.head())

Order items with invalid order_id: 0
Order items with invalid product_id: 0


## 6. Validate Basic Value Ranges 

Before cleaning or analysis, check wehter key numeric fields fall within reasonable ranges. 
To detect impossible values (negative prices, unrealistic ages, invalid quantities, etc).

To validate: 

- `customers.age` -> should be between 0 and `100
- `products.price_usd` and `products.cost_usd` -> must be ≥ 0
- `orders.total_usd` and `orders.subtotal)usd` -> must be ≥ 0
- `order_items.quantity` should be positive
- `events.discount_pct` -> should be between 0 and 100
- `events.qty` -> positive of NaN (only relevant for add_to_cart)

if any values fall outside expected range, investigate before cleaning.


In [35]:
# 1. Check age values between 0 -12-
print("Invalid ages:", customers[(customers["age"] < 0) | (customers["age"] > 120)].shape[0])

# 2. Check product cost/price >= 0
print("Negative product prices:", products[products["price_usd"] < 0].shape[0])
print("Negative product costs:", products[products["cost_usd"] < 0].shape[0])

# 3. Check orders total amounts >= 0
print("Negative subtotal:", orders[orders["subtotal_usd"] < 0].shape[0])
print("Negative total:", orders[orders["total_usd"] <0].shape[0])

# 4. Check order item quantities > 0
print("Invalid quantities:", order_items[order_items["quantity"] <=0].shape[0])

# 5. Check discount percentage range between 0 - 100
print("Discounts outside 0-100:", events[(events["discount_pct"] < 0) | (events["discount_pct"] > 100)].shape[0])

# 6. Check event qty (should be NaN or positive)
print("Invalid event qty:", events[events["qty"] <0].shape[0])

Invalid ages: 0
Negative product prices: 0
Negative product costs: 0
Negative subtotal: 0
Negative total: 0
Invalid quantities: 0
Discounts outside 0-100: 0
Invalid event qty: 0
